In [24]:
## 지번 주소로 위도, 경도 찾기

## aws DB 에 직접 저장

In [ ]:
from geopy.geocoders import Nominatim
from pprint import pprint

In [25]:
import pandas as pd
import numpy as np
import pymysql as my

In [34]:
store_df = pd.read_csv('STORE.csv', encoding='utf-8')
store_df.head()

,LOCAL_CODE
0,(지번) 창동 337-1
1,(지번) 도봉동 470-3
2,(지번) 쌍문동 100-35
3,(지번) 창동 700-27
4,(지번) 쌍문동 395-6


In [35]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10139 entries, 0 to 10138
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   LOCAL_CODE  10139 non-null  object
dtypes: object(1)
memory usage: 79.3+ KB


In [36]:
store = store_df.dropna(axis=0)

In [37]:
store.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10139 entries, 0 to 10138
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   LOCAL_CODE  10139 non-null  object
dtypes: object(1)
memory usage: 158.4+ KB


In [7]:
# service = Nominatim(user_agent = 'South Korea')
# service.geocode('대치동 896-5')
# print(location.latitude, location.longitude)

In [8]:
# for local in store['local']:
#     try:
#         i = ' '.join(local.split(' ')[1:])
#         location = service.geocode(i)
#         print(i, location.latitude, location.longitude)
#     except:
#         print(i)

In [73]:
### DB 다루는 클래스
class DBHelper:
    '''
    맴버변수 : 커넥션 
    '''
    conn = None
    '''
    생성자
    '''
    def __init__(self):
        self.db_init()
    '''
    맴버 함수
    '''
    def db_init(self):
        self.conn = my.connect(
                        host='3.34.111.86',
                        user='root',
                        password='123456',
                        db='project_db',
                        charset='utf8mb4',
                        cursorclass=my.cursors.DictCursor
        )


    def db_free(self):
        if self.conn:
            self.conn.close()


    def db_insertCrawlingData_PLACE(self, local, latitude, longitude):
        with self.conn.cursor() as cursor:
            sql = "INSERT INTO PLACE (LOCAL_CODE, LATITUDE, LONGITUDE) VALUES ( %s, %s, %s )"
            cursor.execute(sql, (local, latitude, longitude)) 
        self.conn.commit()
        

In [74]:
db = DBHelper()

In [72]:
# db.db_free()

In [75]:
place = pd.DataFrame()

service = Nominatim(user_agent = 'South Korea')

In [76]:
for i, local in enumerate(store['LOCAL_CODE']):
    try:
        j = ' '.join(local.split(' ')[1:])
        location = service.geocode(j)
        print(local, location.latitude, location.longitude)
        
        db.db_insertCrawlingData_PLACE(local, location.latitude, location.longitude)
        
        row = {'local' : local , 'latitude' : location.latitude, 'longitude' : location.longitude}
        row = pd.DataFrame(row, index=[i])
        place = place.append(row, ignore_index=True)
        
    except:
        
        db.db_insertCrawlingData_PLACE(local, None, None)
        
        row = {'local' : local , 'latitude' : None, 'longitude' : None}
        row = pd.DataFrame(row, index=[i])
        place = place.append(row, ignore_index=True)

(지번) 창동 337-1 37.29343 127.63346
(지번) 도봉동 470-3 37.6826056 127.0425405
(지번) 쌍문동 100-35 37.65347 127.02715
(지번) 창동 700-27 36.636327 128.1935133
(지번) 쌍문동 395-6 37.65347 127.02715
(지번) 도봉1동 600-4 37.685468549999996 127.0274286535063
(지번) 창동 74-25 36.636327 128.1935133
(지번) 도봉동 596-23 37.6826056 127.0425405
(지번) 도봉동 286-27 37.6826056 127.0425405
(지번) 방학동 436-9 37.66442 127.03333
(지번) 창2동 650-30 37.64166375 127.03680739868314
(지번) 쌍문동 81-74 37.65347 127.02715
(지번) 방학1동 680-14 37.66504505 127.04358593946017
(지번) 방학동 598-1 37.66442 127.03333
(지번) 창동 715-4 36.636327 128.1935133
(지번) 창동 608-142 37.5358868 127.2276072
(지번) 방학동 677-20 37.66442 127.03333
(지번) 창동 659-16 36.636327 128.1935133
(지번) 방학동 669-3 37.66442 127.03333
(지번) 창동 674-29 36.636327 128.1935133
(지번) 창동 635-42 36.636327 128.1935133
(지번) 창동 651-5 35.20637 128.57426
(지번) 도봉동 625-13 37.6826056 127.0425405
(지번) 창동 700-29 36.636327 128.1935133
(지번) 쌍문동 47-2 37.65347 127.02715
(지번) 도봉동 343-2 37.6826056 127.0425405
(지번) 창동 750 36.636327 12

In [14]:
place.tail()

,local,latitude,longitude
406,(지번) 도곡동 542-6,37.487839,127.043857
407,(지번) 신사동 650-16,NaN,NaN
408,(지번) 역삼동 817-22,37.501184,127.038256
409,(지번) 신사동 534-12,37.522321,127.027694
410,(지번) 논현동 183-8,37.513261,127.031563


In [44]:
# place = pd.DataFrame()

# review_lists = ['대치동 896-5']

# for i, local in enumerate(store['LOCAL_CODE']):
#     print(local)

# print(place)

(지번) 창동 337-1
(지번) 도봉동 470-3
(지번) 쌍문동 100-35
(지번) 창동 700-27
(지번) 쌍문동 395-6
(지번) 도봉1동 600-4
(지번) 창동 74-25
(지번) 도봉동 596-23
(지번) 도봉동 286-27
(지번) 방학동 436-9
(지번) 창2동 650-30
(지번) 쌍문동 81-74
(지번) 방학1동 680-14
(지번) 방학동 598-1
(지번) 창동 715-4
(지번) 창동 608-142
(지번) 방학동 677-20
(지번) 창동 659-16
(지번) 방학동 669-3
(지번) 창동 674-29
(지번) 창동 635-42
(지번) 창동 651-5
(지번) 도봉동 625-13
(지번) 창동 700-29
(지번) 쌍문동 47-2
(지번) 도봉동 343-2
(지번) 창동 750
(지번) 도봉동 635
(지번) 창동 269-4
(지번) 도봉동 560-19
(지번) 쌍문동 103-30
(지번) 쌍문동 395-2
(지번) 도봉동 279-13
(지번) 창동 659-1
(지번) 도봉동 597-21
(지번) 창동 26
(지번) 창동 813
(지번) 방학동 669
(지번) 창동 658-49
(지번) 방학동 716-14
(지번) 창동 657-68
(지번) 창동 651-5
(지번) 쌍문동 422-80
(지번) 창동 651-51
(지번) 창동 333
(지번) 창동 651-64
(지번) 창동 715-32
(지번) 방학동 590-2
(지번) 창동 657-111
(지번) 창동 621-31
(지번) 창동 134-2
(지번) 쌍문동 423-17
(지번) 창동 446-4
(지번) 창동 74-25
(지번) 방학동 675-33
(지번) 방학동 664
(지번) 쌍문동 315-354
(지번) 창동 750
(지번) 도봉동 597-20
(지번) 창동 286-10
(지번) 창동 661-80
(지번) 방학동 592-1
(지번) 창동 338
(지번) 방학동 263-75
(지번) 창동 820
(지번) 창동 715-4
(지번) 방학동 670-2
(지번) 창동 4
(지